<a href="https://colab.research.google.com/github/AfonsoPaula/Data-Analysis/blob/main/handling_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Handling Missing Data with Pandas**

In [1]:
import numpy as np
import pandas as pd

## **Pandas utility functions**

Similary to **numpy**, pandas also has a few utility functions to identify and delect null values:

In [2]:
pd.isnull(np.nan)

True

In [3]:
pd.isnull(None)

True

In [4]:
pd.isna(np.nan)

True

In [5]:
pd.isna(None)

True

The opposite ones also exist:

In [6]:
pd.notnull(None)

False

In [7]:
pd.notnull(np.nan)

False

In [8]:
pd.notna(None)

False

In [9]:
pd.notna(np.nan)

False

In [10]:
pd.notnull(3)

True

These functions also work with Series and **DataFrames**:

In [11]:
pd.isnull(pd.Series([1, np.nan, 7]))

0    False
1     True
2    False
dtype: bool

In [12]:
pd.notnull(pd.Series([1, np.nan, 7]))

0     True
1    False
2     True
dtype: bool

In [13]:
pd.isnull(pd.DataFrame({
    'Column A':[1, np.nan, 7],
    'Column B':[np.nan, 2, 3],
    'Column C':[np.nan, 2, np.nan]
}))

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


## **Pandas Operations with Missing Values**

Pandas manages missing values more gracefully than numpy. **nan**s will no longer behave as "viruses", and operation will just ignore them completely:

In [14]:
pd.Series([1, 2, np.nan]).count()

2

In [15]:
pd.Series([1, 2, np.nan]).sum()

3.0

In [16]:
pd.Series([1, 2, np.nan]).mean()

1.5

## **Filtering missing data**

As we saw with numpy, we could combine boolean selection + **pd.isnull** to filter out those **nans** and null values:

In [17]:
s = pd.Series([1, 2, 3, np.nan, np.nan, 4])

In [18]:
pd.notnull(s)

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [19]:
pd.isnull(s)

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [20]:
pd.notnull(s).sum()

4

In [21]:
pd.isnull(s).sum()

2

In [22]:
s[pd.notnull(s)]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

But both **notnull** and **isnull** are also methods of **Series** and **DataFrames**, so we could use it that way:

In [23]:
s.isnull()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [24]:
s.notnull()

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [25]:
s[s.notnull()]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

## **Dropping null values**

Boolean selection + **notnull()** seems alittle bit verbose and repetitive. In this case, we can use the **dropna** method:

In [26]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

In [27]:
s.dropna()

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

## **Dropping null values on Dataframes**
We saw how simple it is to drop **na** with a Series. But with **DataFrames**, there will be a few more things to consider, because we can't drop single values. We can only drop entire columns or rows. Let's start with a sample **DataFrame**:


In [28]:
df = pd.DataFrame({
    'Column A': [1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110],
})

In [29]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [30]:
df.shape

(4, 4)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Column A  2 non-null      float64
 1   Column B  3 non-null      float64
 2   Column C  3 non-null      float64
 3   Column D  4 non-null      int64  
dtypes: float64(3), int64(1)
memory usage: 256.0 bytes


In [32]:
df.isnull()

,Column A,Column B,Column C,Column D
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,True,True,False,False


In [33]:
df.isnull().sum()

Column A    2
Column B    1
Column C    1
Column D    0
dtype: int64

The default **dropna** behavior will drop all the rows in which *any* null value is present:

In [34]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In this case we're dropping **rows**. Rows containing null values are dropped from the DF. We can also use the **axis** parameter to drop columns containing null values:

In [35]:
df.dropna(axis=1) # axis='columns' also works

,Column D
0,5
1,8
2,34
3,110


In this case, any row or column that contains **at least** one null value will be dropped. Which can be, depending on the case, too extreme. We can control this behavior with the **how** parameter. Can be either **'any'** or **'all'**:

In [36]:
df2 = pd.DataFrame({
    'Column A': [1, np.nan, 30],
    'Column B': [2, np.nan, 31],
    'Column C': [np.nan, np.nan, 100]
})

In [37]:
df2

,Column A,Column B,Column C
0,1.0,2.0,NaN
1,NaN,NaN,NaN
2,30.0,31.0,100.0


In [38]:
df2.dropna(how='all')

,Column A,Column B,Column C
0,1.0,2.0,NaN
2,30.0,31.0,100.0


In [39]:
df2.dropna(how='any') # default behavior

,Column A,Column B,Column C
2,30.0,31.0,100.0


We can also use the **thresh** parameter to indicate a *threshold* (a minimum number) of non-null values for the row/column to be kepts:

In [40]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [41]:
df.dropna(thresh=3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [42]:
df.dropna(thresh=3, axis='columns')

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


## **Filling null values**

Sometimes instead than dropping the null values, we might need to replace them with some other values. This highly depends on the context and the dataset we're currently working. Sometimes a **nan** can be replaced with a **0**, sometimes it can be replaced with the **mean** of the sample, and some other times we can take the closest value. Again, it depends of the context.

In [43]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

**Filling nulls with a arbitrary value**

In [44]:
s.fillna(0)

0    1.0
1    2.0
2    3.0
3    0.0
4    0.0
5    4.0
dtype: float64

In [45]:
s.fillna(s.mean())

0    1.0
1    2.0
2    3.0
3    2.5
4    2.5
5    4.0
dtype: float64

In [46]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

**Filling nulls with contiguous (close) values**

The **method** argument is used to fill null values with other values close to that null one:

In [47]:
s.fillna(method='ffill')

0    1.0
1    2.0
2    3.0
3    3.0
4    3.0
5    4.0
dtype: float64

In [48]:
s.fillna(method='bfill')

0    1.0
1    2.0
2    3.0
3    4.0
4    4.0
5    4.0
dtype: float64

This can still leave null values at the extremes of the Series/DataFrames:

In [49]:
pd.Series([np.nan, 3, np.nan, 9]).fillna(method='ffill')

0    NaN
1    3.0
2    3.0
3    9.0
dtype: float64

In [50]:
pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method='bfill')

0    1.0
1    3.0
2    3.0
3    NaN
4    NaN
dtype: float64

## **Filling null values on DataFrames**

The **fillna** method also works on **DataFrames**, and it works similarly. The main differences are that you can specify the **axis** (as usual, rows and columns) to use to fill the values (specially for methods) and that you have more control on the values passed:

In [51]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [52]:
df.fillna({'Column A': 0, 'Column B': 99, 'Column C': df['Column C'].mean()})

,Column A,Column B,Column C,Column D
0,1.0,2.0,47.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,99.0,100.0,110


In [53]:
df.fillna(method='ffill', axis=0)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [54]:
df.fillna(method='ffill', axis=1)

,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0


## **Checking if there are NAs**

The question is: Does this **Series** or **DataFrames** contain any missing value? The answer should be yes or no: **True** or **False**. How can we verify it?

**Example 1: Checking the length**

If there are missing values, **s.dropna()** will have less elements than s:

In [55]:
s.dropna().count()

4

In [56]:
missing_values = len(s.dropna()) != len(s)
missing_values

True

There's also a **count** method, that excludes **nan** from its result:

In [57]:
len(s)

6

In [58]:
s.count()

4

So we could just do:

In [59]:
missing_values = s.count() != len(s)
missing_values

True

**More Pythonic Solution - *any***

The methods **any** and **all** check if either there's **any** True value in a Series or **all** the values are **True**. They works in the same way as in Python:

In [60]:
pd.Series([True, False, False]).any()

True

In [61]:
pd.Series([True, False, False]).all()

False

In [62]:
pd.Series([True, True, True]).all()

True

The **isnull()** method returned a Boolean **Series** with **True** values wherever there was a **nan**:

In [63]:
s.isnull()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

So we can just use the **any** method with the boolean array returned:

In [64]:
pd.Series([1, np.nan]).isnull().any()

True

In [65]:
pd.Series([1, 2]).isnull().any()

False

In [66]:
s.isnull().any()

True

A more strict version would check the **values** of the Series:

In [67]:
s.isnull().values

array([False, False, False,  True,  True, False])

In [68]:
s.isnull().values.any()

True